In [1]:
# Loading pandas
import pandas as pd

# Loading date wrangling package
from datetime import datetime

# Reading the input data
d = pd.read_csv("D:\Data Science\AC_Data.csv")

In [2]:
d = pd.read_csv("D:\Data Science\AC_Data.csv")
# rename the timeseries
d = d.rename(columns={'0':"Datetime"}, inplace = False)
d.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Datetime'], axis='columns', inplace=True)
d.head()

,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
0,7.518632,8.788315,0.0,0.0,2.617045,4.079041,2.782276,4.624447,5.222060,2.151238,1.585072,0.560373,3.142941,2.749470,5.417774,4.113460,3.305072,6.735981
1,7.518632,8.788315,0.0,0.0,2.617045,4.079041,2.782276,4.624447,5.222060,2.151238,1.585072,0.560373,3.142941,2.749470,5.417774,4.113460,3.305072,6.735981
2,7.426114,8.940615,0.0,0.0,2.581625,3.781231,2.529366,5.057423,5.349465,2.414715,2.168184,1.818730,3.085110,2.720484,3.302422,3.986483,3.220588,6.379500
3,7.426114,8.940615,0.0,0.0,2.581625,3.781231,2.529366,5.057423,5.349465,2.414715,2.168184,1.818730,3.085110,2.720484,3.302422,3.986483,3.220588,6.379500
4,7.052986,9.161103,0.0,0.0,2.592095,3.800127,2.332304,6.322521,3.995392,2.237114,3.345624,2.310409,3.132799,2.676861,3.539026,3.797881,3.131560,6.363475


In [3]:
# We then need a function that converts the time series into an X and Y matrices for the deep learning model to start learning.
ts = [1621.0, 1536.0, 1500.0, 1434.0, 1489.0, 1620.0]
X = [
[1621.0, 1536.0, 1500.0], # First three lags
[1536.0, 1500.0, 1434.0], # Second three lags
[1500.0, 1434.0, 1489.0], # Third three lags
]
Y = [1434.0, 1489.0, 1620.0]

In [4]:
import numpy as np

def create_X_Y(ts: list, lag: int) -> tuple:
    """
    A method to create X and Y matrix from a time series list for the training of 
    deep learning models 
    """
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag):
            Y.append(ts[i + lag])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an LSTM input shape 
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, Y

In [5]:
# Deep learning packages
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Defining the number of neurons in the LSTM layer
n_layer = 50

# Defining how many lags will be used in the time series
n_lag = 3

# Defining the model
model = Sequential()
model.add(LSTM(n_layer, activation='relu', input_shape=(n_lag, 1)))
model.add(Dense(1))

# The objective functions which will be minimized is mean squared error (mse)
model.compile(optimizer='adam', loss='mse')

In [6]:
# Data wrangling
import pandas as pd
import numpy as np

# Deep learning: 
from keras.models import Sequential
from keras.layers import LSTM, Dense


class DeepModelTS():
    """
    A class to create a deep time series model
    """
    def __init__(
        self, 
        data: pd.DataFrame, 
        Y_var: str,
        lag: int,
        LSTM_layer_depth: int, 
        epochs=10, 
        batch_size=256,
        train_test_split=0
    ):

        self.data = data 
        self.Y_var = Y_var 
        self.lag = lag 
        self.LSTM_layer_depth = LSTM_layer_depth
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_test_split = train_test_split

    @staticmethod
    def create_X_Y(ts: list, lag: int) -> tuple:
        """
        A method to create X and Y matrix from a time series list for the training of 
        deep learning models 
        """
        X, Y = [], []

        if len(ts) - lag <= 0:
            X.append(ts)
        else:
            for i in range(len(ts) - lag):
                Y.append(ts[i + lag])
                X.append(ts[i:(i + lag)])

        X, Y = np.array(X), np.array(Y)

        # Reshaping the X array to an LSTM input shape 
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        return X, Y         

    def create_data_for_NN(
        self,
        use_last_n=None
        ):
        """
        A method to create data for the neural network model
        """
        # Extracting the main variable we want to model/forecast
        y = self.data[self.Y_var].tolist()

        # Subseting the time series if needed
        if use_last_n is not None:
            y = y[-use_last_n:]

        # The X matrix will hold the lags of Y 
        X, Y = self.create_X_Y(y, self.lag)

        # Creating training and test sets 
        X_train = X
        X_test = []

        Y_train = Y
        Y_test = []

        if self.train_test_split > 0:
            index = round(len(X) * self.train_test_split)
            X_train = X[:(len(X) - index)]
            X_test = X[-index:]     
            
            Y_train = Y[:(len(X) - index)]
            Y_test = Y[-index:]

        return X_train, X_test, Y_train, Y_test

    def LSTModel(self):
        """
        A method to fit the LSTM model 
        """
        # Getting the data 
        X_train, X_test, Y_train, Y_test = self.create_data_for_NN()

        # Defining the model
        model = Sequential()
        model.add(LSTM(self.LSTM_layer_depth, activation='relu', input_shape=(self.lag, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # Defining the model parameter dict 
        keras_dict = {
            'x': X_train,
            'y': Y_train,
            'batch_size': self.batch_size,
            'epochs': self.epochs,
            'shuffle': False
        }

        if self.train_test_split > 0:
            keras_dict.update({
                'validation_data': (X_test, Y_test)
            })

        # Fitting the model 
        model.fit(
            **keras_dict
        )

        # Saving the model to the class 
        self.model = model

        return model

    def predict(self) -> list:
        """
        A method to predict using the test data used in creating the class
        """
        yhat = []

        if(self.train_test_split > 0):
        
            # Getting the last n time series 
            _, X_test, _, _ = self.create_data_for_NN()        

            # Making the prediction list 
            yhat = [y[0] for y in self.model.predict(X_test)]

        return yhat

    def predict_n_ahead(self, n_ahead: int):
        """
        A method to predict n time steps ahead
        """    
        X, _, _, _ = self.create_data_for_NN(use_last_n=self.lag)        

        # Making the prediction list 
        yhat = []

        for _ in range(n_ahead):
            # Making the prediction
            fc = self.model.predict(X)
            yhat.append(fc)

            # Creating a new input matrix for forecasting
            X = np.append(X, fc)

            # Ommiting the first variable
            X = np.delete(X, 0)

            # Reshaping for the next iteration
            X = np.reshape(X, (1, len(X), 1))

        return yhat    

In [7]:
deep_learner = DeepModelTS(
data = d,
Y_var = 'AC 18',
lag = 6,
LSTM_layer_depth = 50,
epochs = 10,
batch_size = 256,
train_test_split = 0.15
)

In [8]:
#Fitting the model:
model = deep_learner.LSTModel()

Epoch 1/10
292/292 [==============================] - 2s 6ms/step - loss: 2.4574 - val_loss: 0.2120
Epoch 2/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2792 - val_loss: 0.1653
Epoch 3/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2430 - val_loss: 0.1505
Epoch 4/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2322 - val_loss: 0.1418
Epoch 5/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2263 - val_loss: 0.1409
Epoch 6/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2251 - val_loss: 0.1420
Epoch 7/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2243 - val_loss: 0.1415
Epoch 8/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2237 - val_loss: 0.1410
Epoch 9/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2234 - val_loss: 0.1404
Epoch 10/10
292/292 [==============================] - 1s 5ms/step - loss: 0.2230 - val_loss: 0.1395

In [9]:
#Training the model with more lags
deep_learner = DeepModelTS(
data = d,
Y_var = 'AC 18',
lag = 24, # 24 past hours are used
LSTM_layer_depth = 50,
epochs = 10,
batch_size = 256,
train_test_split = 0.15
)
model = deep_learner.LSTModel()

Epoch 1/10
292/292 [==============================] - 6s 22ms/step - loss: 1.7799 - val_loss: 0.2168
Epoch 2/10
292/292 [==============================] - 6s 21ms/step - loss: 0.2716 - val_loss: 0.1487
Epoch 3/10
292/292 [==============================] - 7s 25ms/step - loss: 0.2410 - val_loss: 0.1382
Epoch 4/10
292/292 [==============================] - 8s 27ms/step - loss: 0.2265 - val_loss: 0.1369
Epoch 5/10
292/292 [==============================] - 8s 27ms/step - loss: 0.2229 - val_loss: 0.1385
Epoch 6/10
292/292 [==============================] - 7s 24ms/step - loss: 0.2218 - val_loss: 0.1386
Epoch 7/10
292/292 [==============================] - 7s 25ms/step - loss: 0.2204 - val_loss: 0.1391
Epoch 8/10
292/292 [==============================] - 7s 24ms/step - loss: 0.2197 - val_loss: 0.1388
Epoch 9/10
292/292 [==============================] - 7s 25ms/step - loss: 0.2193 - val_loss: 0.1386
Epoch 10/10
292/292 [==============================] - 7s 25ms/step - loss: 0.2186 - val_lo

In [10]:
# Defining the lag that we used for training of the model 
lag_model = 24
# Getting the last period
ts = d['AC 18'].tail(lag_model).values.tolist()
# Creating the X matrix for the model
X, _ = deep_learner.create_X_Y(ts, lag=lag_model)
# Getting the forecast
yhat = model.predict(X)

In [19]:
import matplotlib.pyplot as plt
yhat = deep_learner.predict()
# Constructing the forecast dataframe
fc = d.tail(len(yhat)).copy()
fc.reset_index(inplace=True)
fc['forecast'] = yhat
fc.tail()

,index,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18,forecast
13167,87835,0.000000,6.122385,2.192198,2.083315,1.046250,0.000000,3.668421,3.006311,3.614301,1.860847,5.019769,3.154221,3.648026,2.439526,4.062719,1.854352,3.027539,7.259989,7.334387
13168,87836,0.000000,6.122385,2.192198,2.083315,1.046250,0.000000,3.668421,3.006311,3.614301,1.860847,5.019769,3.154221,3.648026,2.439526,4.062719,1.854352,3.027539,7.259989,7.258224
13169,87837,1.209176,6.152532,2.211421,0.000000,1.448103,0.000000,3.151248,2.871690,3.417942,2.185493,5.174168,2.772349,3.786657,2.060023,4.057525,1.870886,2.991946,7.134647,7.247559
13170,87838,1.209176,6.152532,2.211421,0.000000,1.448103,0.000000,3.151248,2.871690,3.417942,2.185493,5.174168,2.772349,3.786657,2.060023,4.057525,1.870886,2.991946,7.134647,7.131079
13171,87839,1.430442,6.078281,1.891162,0.000000,1.577491,4.364239,3.126339,3.287224,3.986978,2.013912,4.821829,3.363855,3.380110,1.949240,4.026304,1.862166,3.009434,7.179719,7.121943
